In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import os

# 모듈이 있는 디렉토리의 절대 경로 가져오기
module_dir = '/content/drive/MyDrive/pythondeep'

# Python 경로에 디렉토리 추가
sys.path.append(module_dir)

아래 코드는 데이터 shape 확인용

In [3]:
import torch
import numpy as np
from scipy.io import loadmat
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from model_ import boReEsNet, ReEsNet

rs_file_path = '/content/drive/My Drive/pythondeep/data/RS.mat'  # RS 파일 경로(정답)
rs_data = loadmat(rs_file_path)
RS_real = torch.tensor(rs_data['RS'].real, dtype=torch.float32) # 실수부
# RS_imag = torch.tensor(1j * rs_data['RS'].imag, dtype=torch.complex64)
RS_imag = torch.tensor(rs_data['RS'].imag, dtype=torch.float32) # 허수부
RS_reference = torch.stack([RS_real, RS_imag], dim=-1)  #  72x14 여야함
RS_reference = RS_reference[1:73, :, :]
print(RS_reference.shape)
RS_reference = RS_reference.permute(2,0,1)
print(RS_reference.shape)

torch.Size([72, 14, 2])
torch.Size([2, 72, 14])


In [4]:
import numpy as np
from scipy.io import loadmat
import torch

test_data_path = '/content/drive/My Drive/pythondeep/data/test_data_snr.mat'
mat_data = np.array(loadmat(test_data_path)['test_data_snr']).T
print(mat_data.shape)
data_snr = mat_data[:, 0]
print(data_snr.shape)
for i in range(len(data_snr)):
  data_snr[i] = torch.tensor(data_snr[i]).permute(2,0,1)
  # print(data_snr[i].shape)
print(data_snr[0].shape)

(5000, 4)
(5000,)
torch.Size([2, 24, 2])


In [5]:
# 테스트 데이터셋 클래스 정의
class TestDataset(Dataset):
    def __init__(self, data_path, snr):
        mat_data = np.array(loadmat(test_data_path)['test_data_snr']).T # 5000x4
        data_snr = mat_data[:, snr]
        self.data = [torch.tensor(item).permute(2, 0, 1) for item in data_snr]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature_signal = self.data[idx]
        feature_signal = feature_signal.float()
        return feature_signal


# 테스트 데이터셋 및 DataLoader 설정
test_data_path = '/content/drive/My Drive/pythondeep/data/test_data_snr.mat'  # test_data_snr 파일 경로


snr_values = [0,1,2,3]
testdatasets = []

for snr in snr_values:
    test_dataset = TestDataset(test_data_path, snr=snr)
    testdatasets.append(test_dataset)

batch_size = 128
test_loaders = []

# 각 test_dataset에 대해 DataLoader를 생성하여 test_loaders에 추가
for test_dataset in testdatasets:
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    test_loaders.append(test_loader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from model_ import boReEsNet, ReEsNet  # 모델 정의가 별도 파일

model1 = boReEsNet(input_channels=2, num_filters=8, num_classes=2)  # 모델의 구조 정의, 초기화
model2 = ReEsNet(input_channels=2, num_filters=8, num_classes=2)

model1_path = '/content/drive/MyDrive/pythondeep/data/boReEsNet12.pth'
model1.load_state_dict(torch.load(model1_path))  # 학습된 모델의 가중치 로드
model1.to(device)
model1.eval()

model2_path = '/content/drive/MyDrive/pythondeep/data/ReEsNet12.pth'
model2.load_state_dict(torch.load(model2_path))  # 학습된 모델의 가중치 로드
model2.to(device)
model2.eval()

def test_model(model, test_loaders, snr_values):
    mse_results = {}
    with torch.no_grad():
        for snr_index, test_loader in enumerate(test_loaders):
            mse_results[snr_index] = 0
            for batch in test_loader:
                data_s = batch.to(device)
                outputs = model(data_s)
                RS = RS_reference.to(device)
                mse_value = ((outputs - RS) ** 2).mean()  # MSE 계산
                mse_results[snr_index] += mse_value.item()
            mse_results[snr_index] /= len(test_loader)  # 평균 MSE 계산
            print(f'SNR: {snr_values[snr_index]}, MSE: {mse_results[snr_index]}')
    return mse_results

results = {}
# 모델 1 테스트_boReEsNet
print("boReEsNet")
results["model1"] = test_model(model1, test_loaders, snr_values)

# 모델 2 테스트_ReEsNet
print("ReEsNet")
results["model2"] = test_model(model2, test_loaders, snr_values)





<ipython-input-5-07b4c023ddab>:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load(model1_path))  # 학습된 모델의 가중치 로드
<ipython-input-5-07b4c023d

boReEsNet
SNR: 0, MSE: 0.8670875325798988
SNR: 1, MSE: 0.6877617470920085
SNR: 2, MSE: 0.6810987517237663
SNR: 3, MSE: 0.6943194016814231
ReEsNet
SNR: 0, MSE: 0.8663461059331894
SNR: 1, MSE: 0.7344216275960207
SNR: 2, MSE: 0.7220889836549759
SNR: 3, MSE: 0.7337856635451316


In [6]:
import torch
import torch.nn as nn

class TestDataset(Dataset):
    def __init__(self, data_path, snr):
        mat_data = np.array(loadmat(test_data_path)['test_data_snr']).T # 5000x4
        data_snr = mat_data[:, snr]
        self.data = [torch.tensor(item).permute(2, 0, 1) for item in data_snr]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature_signal = self.data[idx]
        feature_signal = feature_signal.float()
        return feature_signal


# 테스트 데이터셋 및 DataLoader 설정
test_data_path = '/content/drive/My Drive/pythondeep/data/test_data_snr.mat'  # test_data_snr 파일 경로
snr_values = [0,1,2,3]
testdatasets = []

for snr in snr_values:
    test_dataset = TestDataset(test_data_path, snr=snr)
    testdatasets.append(test_dataset)

batch_size = 128
test_loaders = []

# 각 test_dataset에 대해 DataLoader를 생성하여 test_loaders에 추가
for test_dataset in testdatasets:
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    test_loaders.append(test_loader)

# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 3. 모델 초기화 및 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from model_ import boReEsNet, ReEsNet  # 모델 정의가 별도 파일에 있다고 가정

model1 = boReEsNet(input_channels=2, num_filters=8, num_classes=2)  # 모델의 구조 정의, 초기화
model2 = ReEsNet(input_channels=2, num_filters=8, num_classes=2)

model1_path = '/content/drive/MyDrive/pythondeep/data/boReEsNet12.pth'
model1.load_state_dict(torch.load(model1_path))  # 학습된 모델의 가중치 로드
model1.to(device)
model1.eval()

model2_path = '/content/drive/MyDrive/pythondeep/data/ReEsNet12.pth'
model2.load_state_dict(torch.load(model2_path))  # 학습된 모델의 가중치 로드
model2.to(device)
model2.eval()

def test_model_with_loss(model, test_loaders, snr_values, criterion):
    loss_results = {}
    with torch.no_grad():
        for snr_index, test_loader in enumerate(test_loaders):
            loss_results[snr_index] = 0
            for batch in test_loader:
                data_s = batch.to(device)  # 입력 데이터
                outputs = model(data_s)  # 모델 출력
                RS = RS_reference.to(device)  # 정답 데이터
                loss_value = criterion(outputs, RS)  # 손실 함수 계산
                loss_results[snr_index] += loss_value.item()
            loss_results[snr_index] /= len(test_loader)  # 평균 Loss 계산
            print(f'SNR: {snr_values[snr_index]}, Loss: {loss_results[snr_index]}')
    return loss_results

# 모델 학습 시 사용된 손실 함수
criterion = nn.MSELoss()

# boReEsNet
print("boReEsNet")
results["model1_loss"] = test_model_with_loss(model1, test_loaders, snr_values, criterion)

# ReEsNet
print("ReEsNet")
results["model2_loss"] = test_model_with_loss(model2, test_loaders, snr_values, criterion)



<ipython-input-6-73973eb89f68>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load(model1_path))  # 학습된 모델의 가중치 로드
<ipython-input-6-73973eb89

boReEsNet


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([2, 72, 14])) that is different to the input size (torch.Size([8, 2, 72, 14])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


SNR: 0, Loss: 0.8670875310897828
SNR: 1, Loss: 0.6877617500722408
SNR: 2, Loss: 0.6810987517237663
SNR: 3, Loss: 0.6943194016814231
ReEsNet
SNR: 0, Loss: 0.8663461089134217
SNR: 1, Loss: 0.7344216261059046
SNR: 2, Loss: 0.7220889836549759
SNR: 3, Loss: 0.7337856650352478
